<a href="https://colab.research.google.com/github/ashouy/Aprendizado-de-Maquina/blob/main/Atividade02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Base para predição do custo de uma consulta médica

> ## Metadata
* O valor da consulta no final está em rúpia indiana IRN

### **Apresentação da base**

In [72]:
import pandas as pd

consultas = pd.read_excel('https://github.com/ashouy/Aprendizado-de-Maquina/blob/main/bases/consultation_fee/train.xlsx?raw=true')
consultas

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Fees
0,"BHMS, MD - Homeopathy",24 years experience,100%,"Kakkanad, Ernakulam",Homeopath,"100% 16 Feedback Kakkanad, Ernakulam",100
1,"BAMS, MD - Ayurveda Medicine",12 years experience,98%,"Whitefield, Bangalore",Ayurveda,"98% 76 Feedback Whitefield, Bangalore",350
2,"MBBS, MS - Otorhinolaryngology",9 years experience,NaN,"Mathikere - BEL, Bangalore",ENT Specialist,NaN,300
3,"BSc - Zoology, BAMS",12 years experience,NaN,"Bannerghatta Road, Bangalore",Ayurveda,"Bannerghatta Road, Bangalore ₹250 Available on...",250
4,BAMS,20 years experience,100%,"Keelkattalai, Chennai",Ayurveda,"100% 4 Feedback Keelkattalai, Chennai",250
...,...,...,...,...,...,...,...
5956,"MBBS, MS - ENT",19 years experience,98%,"Basavanagudi, Bangalore",ENT Specialist,"98% 45 Feedback Basavanagudi, Bangalore",300
5957,MBBS,33 years experience,NaN,"Nungambakkam, Chennai",General Medicine,NaN,100
5958,MBBS,41 years experience,97%,"Greater Kailash Part 2, Delhi",General Medicine,"97% 11 Feedback Greater Kailash Part 2, Delhi",600
5959,"MBBS, MD - General Medicine",15 years experience,90%,"Vileparle West, Mumbai",General Medicine,General Medical Consultation Viral Fever Treat...,100


### **Tratamento**


#### Seleção de colunas

>* seleção de colunas

In [73]:
## verificando as colunas que tem valores não definidos
hasNan = consultas.isnull().sum()
print(hasNan)

Qualification            0
Experience               0
Rating                3302
Place                   25
Profile                  0
Miscellaneous_Info    2620
Fees                     0
dtype: int64


In [74]:
## verificando quantos valores possíveis a coluna "qualificationn tem"
consultas["Qualification"].describe()

count     5961
unique    1420
top        BDS
freq       488
Name: Qualification, dtype: object

In [75]:
consultas["Profile"].describe()

count        5961
unique          6
top       Dentist
freq         1397
Name: Profile, dtype: object

>* A coluna Misscellaneous_info não apresenta dados constantes, ou que respeitam algum padrão, impossibilitando de realizar alguma conversão para dado numérico, e dificultando a análize

>* A coluna "rating" tem 3302 dados faltantes, mais da metade do exemplos, não há possibilidade de preencher com médias ou valores fixos como na coluna "place", que tem apenas 25 exemplos com valores não definidos

>* A coluna "qualification" apresenta 1420 possibilidades, podem dizer a mesma coisa, pois existem exemplos que tem a qualificação diferente porém o perfil ("profile") igual. Podemos observar acima que a coluna "Qualification" tem 1420 valores unicos e a "Profile" tem apenas 6.

>* Conclusão: serão removidas as coluans "rating", "miscellaneous_Info" e "Profile"

In [76]:
consultas = consultas[["Experience",	"Place", "Profile",	"Fees"]]
consultas

,Experience,Place,Profile,Fees
0,24 years experience,"Kakkanad, Ernakulam",Homeopath,100
1,12 years experience,"Whitefield, Bangalore",Ayurveda,350
2,9 years experience,"Mathikere - BEL, Bangalore",ENT Specialist,300
3,12 years experience,"Bannerghatta Road, Bangalore",Ayurveda,250
4,20 years experience,"Keelkattalai, Chennai",Ayurveda,250
...,...,...,...,...
5956,19 years experience,"Basavanagudi, Bangalore",ENT Specialist,300
5957,33 years experience,"Nungambakkam, Chennai",General Medicine,100
5958,41 years experience,"Greater Kailash Part 2, Delhi",General Medicine,600
5959,15 years experience,"Vileparle West, Mumbai",General Medicine,100


### Alterar tipo das colunas

In [77]:
#mudar tipo da experiencia
exp_column_int = consultas['Experience'].str[:2].astype(int)
consultas['Experience'] = exp_column_int
consultas["Experience"].unique()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


array([24, 12,  9, 20,  8, 42, 10, 14, 23,  5,  7, 21, 19, 41, 31, 25, 16,
       15, 11, 22, 38,  6, 36, 13, 18, 46, 40, 49, 33,  0, 27, 30, 52, 60,
       26, 35, 34, 39, 32, 17, 28, 58,  4, 45, 44,  3, 37, 50, 29, 43, 54,
       66,  2, 51, 47, 48, 53, 56, 61, 59, 57, 55, 65, 64])

In [78]:
consultas_base = consultas
consultas_base.head()

,Experience,Place,Profile,Fees
0,24,"Kakkanad, Ernakulam",Homeopath,100
1,12,"Whitefield, Bangalore",Ayurveda,350
2,9,"Mathikere - BEL, Bangalore",ENT Specialist,300
3,12,"Bannerghatta Road, Bangalore",Ayurveda,250
4,20,"Keelkattalai, Chennai",Ayurveda,250


In [79]:
## mudar tipo do profile

profile_types = pd.get_dummies(consultas['Profile'], prefix='spec_')
profile_types

,spec__Ayurveda,spec__Dentist,spec__Dermatologists,spec__ENT Specialist,spec__General Medicine,spec__Homeopath
0,0,0,0,0,0,1
1,1,0,0,0,0,0
2,0,0,0,1,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0
...,...,...,...,...,...,...
5956,0,0,0,1,0,0
5957,0,0,0,0,1,0
5958,0,0,0,0,1,0
5959,0,0,0,0,1,0


In [80]:
# Juntar profile dummies com dataset
consultas = profile_types.join(consultas.drop(columns=["Profile"]))
consultas.head()



,spec__Ayurveda,spec__Dentist,spec__Dermatologists,spec__ENT Specialist,spec__General Medicine,spec__Homeopath,Experience,Place,Fees
0,0,0,0,0,0,1,24,"Kakkanad, Ernakulam",100
1,1,0,0,0,0,0,12,"Whitefield, Bangalore",350
2,0,0,0,1,0,0,9,"Mathikere - BEL, Bangalore",300
3,1,0,0,0,0,0,12,"Bannerghatta Road, Bangalore",250
4,1,0,0,0,0,0,20,"Keelkattalai, Chennai",250


In [81]:
consultas_base = consultas
consultas_base.head()

,spec__Ayurveda,spec__Dentist,spec__Dermatologists,spec__ENT Specialist,spec__General Medicine,spec__Homeopath,Experience,Place,Fees
0,0,0,0,0,0,1,24,"Kakkanad, Ernakulam",100
1,1,0,0,0,0,0,12,"Whitefield, Bangalore",350
2,0,0,0,1,0,0,9,"Mathikere - BEL, Bangalore",300
3,1,0,0,0,0,0,12,"Bannerghatta Road, Bangalore",250
4,1,0,0,0,0,0,20,"Keelkattalai, Chennai",250


>* O local específico da consulta varia muito, e acreditamos que dependendo de onde a consulta for feita, o preço será diferente. Como existem diversos "bairros", nos restringimos as cidades. Para isso fizemos uma função para retornar apenas as cidades

In [82]:
# Alterando tipo da coluna "Place"

#funcão para retornar apenas a cidade
def convert_place_to_city(places):
  only_city = str(places)
  if ", " in only_city:
    pop = only_city.split(', ')
    return pop[1]
  else:
    return 'Bangalore'
    
## mais a frente poderemos perceber que Bangalore é a cidade mais presente, com isso colocaremos os 25
## exemplos que não tem cidade definida para ser igual a "bangalore", pois há um probabilidade maior de acertar,
## além de que apenas 25 num dataset com 5960 exemplos não terão impacto relevante.


In [83]:
# Aplicando a função
novaColunaPlace= consultas['Place'].apply(convert_place_to_city)
novaColunaPlace


0       Ernakulam
1       Bangalore
2       Bangalore
3       Bangalore
4         Chennai
          ...    
5956    Bangalore
5957      Chennai
5958        Delhi
5959       Mumbai
5960        Delhi
Name: Place, Length: 5961, dtype: object

In [86]:
novaColunaPlace.value_counts()


Bangalore             1284
Mumbai                1219
Delhi                 1184
Hyderabad              951
Chennai                855
Coimbatore             228
Ernakulam              153
Thiruvananthapuram      86
Sector 5                 1
Name: Place, dtype: int64

>* como podemos ver acima, Bangalore é a cidade mais presente, mesmo sem os 25 exemplos adicionados a ela

In [88]:
## Convertendo para int (dummies)
dummies_places = pd.get_dummies(novaColunaPlace, prefix='city_')
dummies_places

,city__Bangalore,city__Chennai,city__Coimbatore,city__Delhi,city__Ernakulam,city__Hyderabad,city__Mumbai,city__Sector 5,city__Thiruvananthapuram
0,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
5956,1,0,0,0,0,0,0,0,0
5957,0,1,0,0,0,0,0,0,0
5958,0,0,0,1,0,0,0,0,0
5959,0,0,0,0,0,0,1,0,0


In [ ]:
consultas = dummies_places.join(consultas.drop(columns=["Place"]))

In [93]:
consultas.head()

,city__Bangalore,city__Chennai,city__Coimbatore,city__Delhi,city__Ernakulam,city__Hyderabad,city__Mumbai,city__Sector 5,city__Thiruvananthapuram,spec__Ayurveda,spec__Dentist,spec__Dermatologists,spec__ENT Specialist,spec__General Medicine,spec__Homeopath,Experience,Fees
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,24,100
1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,12,350
2,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,9,300
3,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,12,250
4,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,20,250


In [94]:
consultas_base = consultas
consultas_base

,city__Bangalore,city__Chennai,city__Coimbatore,city__Delhi,city__Ernakulam,city__Hyderabad,city__Mumbai,city__Sector 5,city__Thiruvananthapuram,spec__Ayurveda,spec__Dentist,spec__Dermatologists,spec__ENT Specialist,spec__General Medicine,spec__Homeopath,Experience,Fees
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,24,100
1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,12,350
2,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,9,300
3,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,12,250
4,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,20,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5956,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,19,300
5957,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,33,100
5958,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,41,600
5959,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,15,100


In [95]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [99]:
%cd /content/drive/MyDrive/Colab\ Notebooks

/content/drive/MyDrive/Colab Notebooks


In [100]:
consultas_base.to_csv('./consultasTratadas.csv', index = False,)
#df.to_csv (r'C:\Users\Ron\Desktop\export_dataframe.csv', index = False, header=True)

# Aplicação dos algoritmos

In [101]:
base_tratada = pd.read_csv('https://raw.githubusercontent.com/ashouy/Aprendizado-de-Maquina/main/bases/consultation_fee/consultasTratadas.csv')
base_tratada

,city__Bangalore,city__Chennai,city__Coimbatore,city__Delhi,city__Ernakulam,city__Hyderabad,city__Mumbai,city__Sector 5,city__Thiruvananthapuram,spec__Ayurveda,spec__Dentist,spec__Dermatologists,spec__ENT Specialist,spec__General Medicine,spec__Homeopath,Experience,Fees
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,24,100
1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,12,350
2,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,9,300
3,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,12,250
4,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,20,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5956,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,19,300
5957,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,33,100
5958,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,41,600
5959,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,15,100
